In [1]:
import sys
import time
sys.path.append('../lib')
from func_MSR import *
import pandas as pd

In [2]:
import numpy as np
from numpy.random import Generator, PCG64 ,MT19937, PCG64DXSM
from scipy.stats import truncnorm
import multiprocessing as mp

In [3]:
import matplotlib.pyplot as plt

In [4]:
path="../data_extract/"
ratid=21012
chunksize=20 #in minutes
rats=[20382,24101,21012,22295,20630,22098,23783,24116]
df=pd.read_json(path+"Rat_"+str(ratid)+"_data_extracted.json");
df.columns

Index(['RAT_ID', 'NeuID', 'N_DID', 'N_GID', 'LOC', 'STAT', 'rSCC', 'bScore',
       'hdScore', 'u_spiketime', 'task', 'U_GID', 'time_range', 'duration',
       'REC_Date', 'X', 'Y', 't', 'HD', 'MDirection', 'Speed', 'pmap'],
      dtype='object')

D.animals(3).sessions(18) contains- id: 104\

D.animals(3).sessions(18).userData.containedNeurons.ids contains: 36 neuron recordings (units)\
D.animals(3).recordings(18).sessions.userData.containedNeurons.ids\

D.animals(3).sessions(18).userData.containedNeurons.locs contains: 20 SUBs and 16 CA1s\
D.animals(3).recordings(18).sessions.userData.containedNeurons.locs

In [5]:
neuron_Dids=[99,105,124,130,132,137,143,148,154,161,165,169,171,176,179,182,186,202,204,207,209,211,213,214,215,219,225,231,239,243,247,251,253,256,259,262]

unit_Gids=[4893,4942,5070,5122,5164,5262,5317,5374,5430,5486,5525,5562,5574,5650,5691,5735,5780,5941,5954,5967,5980,5990,6002,6010,6018,6041,6082,6122,6193,6240,6275,6295,6305,6327,6339,6353]


In [6]:
# Function to find the indices
def find_indices(df, column, items):
    results = []
    for item in items:
        for row_idx, row in df.iterrows():
            if item in row[column]:
                results.append((row_idx, row[column].index(item)))
    return results

In [7]:
#df[df['N_DID'].isin(neuron_Dids)]['U_GID']
NuUnis = find_indices(df[df['N_DID'].isin(neuron_Dids)], 'U_GID', unit_Gids)
#len(NuUnis)

In [8]:
## Checking the duration of the recordings
nu,uni=NuUnis[2];
print(df.iloc[nu].U_GID[uni],df.iloc[nu].duration[uni])

5070 1058.725693503256


In [9]:
## 20 min chunk test
print(MSR(df.iloc[nu].u_spiketime[uni],df.iloc[nu].time_range[uni][0],df.iloc[nu].time_range[uni][-1],200))
#for The spike train of length = chunksize
print(MSR(df.iloc[nu].u_spiketime[uni],df.iloc[nu].time_range[uni][0],df.iloc[nu].time_range[uni][0]+(chunksize*60),200))

(3045, np.float64(0.275639691638745), np.float64(0.2345425242478744), np.float64(0.8758157725390185), np.float64(1.110358296786893), np.float64(0.5524100857602311), np.float64(0.4540778641159537), np.float64(0.5352867424147735), np.float64(13.52137460595856))
(3045, np.float64(0.2756547029473171), np.float64(0.24824343868488202), np.float64(0.8628079098876413), np.float64(1.1110513485725233), np.float64(0.6488886731980871), np.float64(0.4520627130667849), np.float64(0.5328343111828758), np.float64(13.784576676401947))


In [10]:
def truncated_normal_toss(mu, jitter, M, no_of_samples, seed=1244):
    # Initialize the random number generator
    rng = Generator(PCG64DXSM(seed=seed))
    # Calculate the lower and upper bounds for the truncation
    sigma=jitter/(2*np.pi)
    lower, upper = (-jitter - mu) / sigma, (jitter - mu) / sigma
    truncated_normal = truncnorm(lower, upper, loc=mu, scale=sigma)
    # Generate the samples
    samples = truncated_normal.rvs(size=(M, no_of_samples), random_state=rng)
    return samples

In [11]:
jitter=5/1000 #5ms
M=len(df.iloc[nu].u_spiketime[uni])
no_of_samples = 5000
mu=0
samples = truncated_normal_toss(mu, jitter, M, no_of_samples)
#plt.hist(samples[:,np.random.randint(0,no_of_samples)],bins=50)

In [12]:
def worker_function(idx):
    jtdat=df.iloc[nu].u_spiketime[uni] + samples[:,idx]
    return MSR(jtdat,df.iloc[nu].time_range[uni][0],df.iloc[nu].time_range[uni][0]+(chunksize*60),200)

In [13]:
params = range(no_of_samples)  # Example iterable
    
with mp.Pool(4) as pool:
    async_results = []
    
    for param in params:
        async_result = pool.apply_async(worker_function, (param,))
        async_results.append(async_result)
    
    # Retrieve results
    results = [ar.get() for ar in async_results]

In [14]:
np.shape(results)

(5000, 9)